<a href="https://colab.research.google.com/github/nangokosu/ebay_ml/blob/main/Contrastive_Loss_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.2

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 2.8MB/s 


In [ ]:
import keras
from keras.layers import *
import numpy as np
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import color,data,transform,filters,util,restoration
import tensorflow as tf
from keras.models import Model
import efficientnet.keras as efn

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 0
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [ ]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
import tensorflow.keras.backend as K
def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)
    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)
    x = Multiply()([init, se])
    return x

In [ ]:
def create_encoder(input_shape):
  inputs=Input(shape=input_shape)
  enet=efn.EfficientNetB2(include_top=False,weights='imagenet',input_tensor=inputs,pooling=None)
  enet.trainable=False
  outputs=squeeze_excite_block(enet.output,ratio=16)
  outputs=GlobalAveragePooling2D()(outputs)
  outputs=Dropout(0.2)(outputs)
  outputs=Dense(120,activation="relu")(outputs)
  outputs=Dense(50,use_bias=False)(outputs)
  #outputs=BatchNormalization()(outputs)
  model = Model(
      inputs=inputs, outputs=outputs, name='encoder'
    )
  return model

In [ ]:
def euclid_process(encoded_a,encoded_b):
  distance = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([encoded_a, encoded_b])
  return distance

In [ ]:
shape=(32,32,3)
input_1=Input(shape=(32,32,3))
input_2=Input(shape=(32,32,3))
encoder=create_encoder(shape)
encoded_1=encoder(input_1)
encoded_2=encoder(input_2)

31940608/31936256 [==============================] - 1s 0us/step


In [ ]:
# ONLY APPLY IF RETRAINING
# encoder=tf.keras.models.load_model('/content/drive/Shareddrives/eBay ML/contrastive_efficientnet_encoder')
# for layer in encoder.layers[:-11]:
#  layer.trainable=False

In [ ]:
encoded_1=encoder(input_1)
encoded_2=encoder(input_2)

In [ ]:
normalize_1=Lambda(lambda x: K.l2_normalize(x, axis=1))(encoded_1)
normalize_2=Lambda(lambda x: K.l2_normalize(x, axis=1))(encoded_2)

In [ ]:
euclid_dist=euclid_process(normalize_1,normalize_2)
euclid_model=Model([input_1,input_2],euclid_dist)

In [ ]:
lrplateau1=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.20,patience=8)
modelCheckpoint=tf.keras.callbacks.ModelCheckpoint('/content/drive/Shareddrives/eBay ML/contrastive_efficientNet',save_best_only=True)
early_stop=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [ ]:
loss=tfa.losses.ContrastiveLoss(margin=2**0.5)
euclid_model.compile(optimizer="Adam",loss=loss)

In [ ]:
df = pd.read_pickle("/content/drive/Shareddrives/eBay ML/normalized_df_pairs_balanced.pkl")
length = df.shape[0]
df = df.iloc[2*length // 4:3*length // 4]
df["similarity2"]=df.similarity.apply(lambda x:0 if x==-1 else 1).astype('int')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_1=efn.preprocess_input(np.stack(df["image_1"]))
image_2=efn.preprocess_input(np.stack(df["image_2"]))

In [ ]:
euclid_model.fit([image_1,image_2],df.similarity2,callbacks=[lrplateau1,modelCheckpoint,early_stop],epochs=50,validation_split=0.10,batch_size=64)

Epoch 1/50
1387/1387 [==============================] - 399s 276ms/step - loss: 0.3956 - val_loss: 0.4390
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/contrastive_efficientNet/assets
Epoch 2/50
1387/1387 [==============================] - 374s 270ms/step - loss: 0.3711 - val_loss: 0.4321
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/contrastive_efficientNet/assets
Epoch 3/50
1387/1387 [==============================] - 371s 268ms/step - loss: 0.3631 - val_loss: 0.4302
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/contrastive_efficientNet/assets
Epoch 4/50
1387/1387 [==============================] - 356s 257ms/step - loss: 0.3553 - val_loss: 0.4299
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/contrastive_efficientNet/assets
Epoch 5/50
1387/1387 [==============================] - 353s 255ms/step - loss: 0.3510 - val_loss: 0.4309
Epoch 6/50
1387/1387 [==============================] - 35

In [ ]:
tf.keras.models.save_model(encoder,"/content/drive/Shareddrives/eBay ML/contrastive_efficientnet_encoder")

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/contrastive_efficientnet_encoder/assets
